<img src='https://hammondm.github.io/hltlogo1.png' style="float:right">

Linguistics 578<br>
Fall 2024<br>
Hammond

## Things to remember about any homework assignment:

1. For this assignment, you will edit this jupyter notebook and turn it in. Do not turn in pdf files or separate `.py` files.
1. Late work is not accepted.
1. Given the way I grade, you should try to answer *every* question, even if you don't like your answer or have to guess.
1. You may *not* use `python` modules that we have not already used in class.
1. You may certainly talk to your classmates about the assignment, but everybody must turn in *their own* work. It is not acceptable to turn in work that is essentially the same as the work of classmates.
1. All code must run. It doesn't have to be perfect, it may not do all that you want it to do, but it must run without error.
1. Code must run in reasonable time. Assume that if it takes more than *5 minutes* to run (on your machine), that's too long.
1. Please do not add, remove, or copy autograded cells.
1. Make sure to select `restart, run all cells` from the `kernel` menu when you're done and before you turn this in!

***

***my name***: Kathleen Costa

***people I talked to about the assignment***: N/A

***

## Homework #3

Here are the imports. Please do not import anything else.

In [1]:
import graphviz

Here's the last version of the `FSA` class from this week's notebook.

```python
class FSA:
    #initialize with optional sets
    def __init__(self,start={},final={},arcs={}):
        self.start = start
        self.final = final
        self.arcs = arcs
    #get the full list of states
    def states(self):
        states = set()
        for s in self.start: states.add(s)
        for s in self.final: states.add(s)
        for a in self.arcs:
            states.add(a[0])
            states.add(a[2])
        return states
    #display the FSA
    def dot(self):
        dot = graphviz.Digraph(graph_attr={'rankdir':'LR'})
        allstates = self.states()
        nonfinal = allstates.difference(self.final)
        dot.node('s',style='invisible')
        for s in nonfinal: dot.node(str(s),shape='circle')
        for s in self.final:
            dot.node(str(s),shape='doublecircle')
        for s in self.start: dot.edge('s',str(s))
        for a in self.arcs:
            dot.edge(str(a[0]),str(a[2]),label=str(a[1]))
        return dot
    #acceptability form a start state
    def accept(self,s):
        #check all start states
        for state in self.start:
            #exit as soon as one works
            if self._accept(s,state):
                return True
        return False
    #acceptability from any state
    def _accept(self,s,st):
        #exit from the recursion
        if s == '':
            if st in self.final: return True
            else: return False
        #recursive clause
        else:
            for arc in self.arcs:
                if arc[0] == st and s[0] == arc[1]:
                    if self._accept(s[1:],arc[2]):
                        return True
        return False
    def alphabet(self):
        alphabet = set()
        for a in self.arcs:
            alphabet.add(a[1])
        return alphabet
    def isComplete(self):
        states = self.states()
        alphabet = self.alphabet()
        c = {}
        for a in self.arcs:
            if a[1] in c:
                c[a[1]].add(a[0])
            else:
                c[a[1]] = {a[0]}
        for a in c:
            if len(c[a]) < len(states):
                return False
        return True
    def complete(self):
        if self.isComplete():
            print('FSA is already complete')
        else:
            states = self.states()
            alphabet = self.alphabet()
            newstate = 0
            while newstate in states:
                newstate += 1
            c = {}
            for a in self.arcs:
                if a[1] in c:
                    c[a[1]].add(a[0])
                else:
                    c[a[1]] = {a[0]}
            start = self.start.copy()
            final = self.final.copy()
            arcs = set()
            for a in self.arcs:
                arcs.add(a)
            for symbol in alphabet:
                arcs.add((newstate,symbol,newstate))
            for symbol in c:
                if len(c[symbol]) < len(states):
                    for state in states:
                        if state not in c[symbol]:
                            arcs.add((state,symbol,newstate))
            fsa = FSA(
                start=start,
                final=final,
                arcs=arcs
            )
            return fsa
```

1. Tweak the `accept()` and `_accept()` methods so that you can accommodate $\epsilon$-arcs. **This is tricky!** In principle, it's possible to have direct or indirect loops with only $\epsilon$. If you're not careful, you can end up in an infinite loop.

In [2]:
class FSA:
    #initialize with optional sets
    def __init__(self,start={},final={},arcs={}):
        self.start = start
        self.final = final
        self.arcs = arcs
    #get the full list of states
    def states(self):
        states = set()
        for s in self.start: states.add(s)
        for s in self.final: states.add(s)
        for a in self.arcs:
            states.add(a[0])
            states.add(a[2])
        return states
    #display the FSA
    def dot(self):
        dot = graphviz.Digraph(graph_attr={'rankdir':'LR'})
        allstates = self.states()
        nonfinal = allstates.difference(self.final)
        dot.node('s',style='invisible')
        for s in nonfinal: dot.node(str(s),shape='circle')
        for s in self.final:
            dot.node(str(s),shape='doublecircle')
        for s in self.start: dot.edge('s',str(s))
        for a in self.arcs:
            dot.edge(str(a[0]),str(a[2]),label=str(a[1]))
        return dot
    #acceptability form a start state
    def accept(self,s):
        # YOUR CODE HERE
        return any(self._accept(s, start_state, set()) for start_state in self.start)
    #acceptability from any state
    def _accept(self,s,st,zeros):
        '''the zeros argument is meant to be a set that
        includes all states that have been visited by
        epsilon-arcs. It starts out empty and gets
        populated as you use epsilon arcs. Once you hit
        a non-epsilon-arc, it's reset to empty, until you
        hit an epsilon arc again.
        '''
        # YOUR CODE HERE
        if not s:
            return st in self.final
        if st in zeros:
            return False
        zeros.add(st)
        epsilon_states = set([st])
        stack = [st]
        while stack:
            current = stack.pop()
            for arc in self.arcs:
                if arc[0] == current and arc[1] == '':
                    next_state = arc[2]
                    if next_state not in epsilon_states:
                        epsilon_states.add(next_state)
                        stack.append(next_state)
        symbol = s[0]
        rest_string = s[1:]
        for state in epsilon_states:
            for arc in self.arcs:
                if arc[0] == state and arc[1] == symbol:
                    next_state = arc[2]
                    if self._accept(rest_string, next_state, set()):
                        return True
        return False
###
    def alphabet(self):
        alphabet = set()
        for a in self.arcs:
            alphabet.add(a[1])
        return alphabet
    def isComplete(self):
        states = self.states()
        alphabet = self.alphabet()
        c = {}
        for a in self.arcs:
            if a[1] in c:
                c[a[1]].add(a[0])
            else:
                c[a[1]] = {a[0]}
        for a in c:
            if len(c[a]) < len(states):
                return False
        return True
    def complete(self):
        if self.isComplete():
            print('FSA is already complete')
        else:
            states = self.states()
            alphabet = self.alphabet()
            newstate = 0
            while newstate in states:
                newstate += 1
            c = {}
            for a in self.arcs:
                if a[1] in c:
                    c[a[1]].add(a[0])
                else:
                    c[a[1]] = {a[0]}
            start = self.start.copy()
            final = self.final.copy()
            arcs = set()
            for a in self.arcs:
                arcs.add(a)
            for symbol in alphabet:
                arcs.add((newstate,symbol,newstate))
            for symbol in c:
                if len(c[symbol]) < len(states):
                    for state in states:
                        if state not in c[symbol]:
                            arcs.add((state,symbol,newstate))
            fsa = FSA(
                start=start,
                final=final,
                arcs=arcs
            )
            return fsa

In [3]:
fsa = FSA(
    start = {0},
    final = {1},
    arcs = {
        (0,'a',0),
        (0,'',1),
        (1,'b',1)
    }
)
assert fsa.accept('aabb')

In [4]:
assert not fsa.accept('ba')

In [5]:
fsa2 = FSA(
    start = {0},
    final = {1},
    arcs = {
        (0,'a',0),
        (0,'',1),
        (0,'',0),
        (1,'b',1)
    }
)
assert fsa2.accept('aabb')

In [6]:
assert not fsa2.accept('ba')

2. Now that we can accommodate $\epsilon$, let's combine FSAs. To do this, we must first be able to make state names unique. In other words, if we have two FSAs $x$ and $y$ that we want to combine, we need to make sure the state names for the two are distinct. To do this, let's write a function that will convert the state names of an FSA to a sequence of digits beginning at any specified digit

In [7]:
def newstatenames(fsa,n):
    '''generate new state names
    args:
        fsa: an existing fsa
        n: the digit to start from
    returns:
        a new fsa
    '''
    # YOUR CODE HERE
    statemap = {}
    current_state = n
    for state in fsa.states():  
        statemap[state] = current_state
        current_state += 1
    
    new_start = {statemap[s] for s in fsa.start}
    new_final = {statemap[s] for s in fsa.final}
    new_arcs = {(statemap[a[0]], a[1], statemap[a[2]]) for a in fsa.arcs}
    
    new_fsa = FSA(
        start=new_start,
        final=new_final,
        arcs=new_arcs
    )
    
    return new_fsa

In [8]:
fsa3 = FSA(
    start = {33},
    final = {10},
    arcs = {
        (33,'a',33),
        (33,'b',14),
        (14,'a',10),
        (10,'b',10)
    }
)
assert newstatenames(fsa3,4).states() == {5,6,4}

3. Now write a function that takes two FSAs and returns a new FSA that is their concatenation. This new function should call `newstatenames()` and make use of $\epsilon$-arcs.

In [9]:
def concatenate(f1,f2):
    '''return the concatenation of two FSAs
    args:
        f1: the first FSA
        f2: the second FSA
    returns:
        a new FSA that's the concatenation of the two arguments
    '''
    # YOUR CODE HERE
    new_state_mapping = {}
    new_start_state = 0

    for state in f1.states():
        new_state_mapping[state] = new_start_state
        new_start_state += 1

    offset = new_start_state

    for state in f2.states():
        new_state_mapping[state] = new_start_state
        new_start_state += 1

    start_states = f1.start
    final_states_f1 = f1.final
    start_states_f2 = f2.start
    final_states_f2 = f2.final

    arcs = set()

    for arc in f1.arcs:
        arcs.add((new_state_mapping[arc[0]], arc[1], new_state_mapping[arc[2]]))

    for arc in f2.arcs:
        if arc[0] in new_state_mapping and arc[2] in new_state_mapping:
            arcs.add((new_state_mapping[arc[0]], arc[1], new_state_mapping[arc[2]]))

    for final_state in final_states_f1:
        if final_state in new_state_mapping:
            for start_state in start_states_f2:
                if start_state in new_state_mapping:
                    arcs.add((new_state_mapping[final_state], '', new_state_mapping[start_state]))

    return FSA(
        start={new_state_mapping[state] for state in start_states},
        final={new_state_mapping[state] for state in final_states_f2},
        arcs=arcs
    )

In [10]:
fsa4 = FSA(
    start={0},
    final={1},
    arcs={
        (0,'a',0),
        (0,'b',1)
    }
)
fsa5 = FSA(
    start={0},
    final={2},
    arcs={
        (0,'c',1),
        (1,'d',2)
    }
)
fsa6 = concatenate(fsa4,fsa5)
assert fsa6.accept('aabcd')

In [11]:
fsa7 = concatenate(fsa4,fsa4)
assert fsa7.accept('baaab')

In [12]:
fsa8 = concatenate(fsa5,fsa5)
assert not fsa8.accept('cd')

AssertionError: 

4. Let's now write a function that computes the $\epsilon$-closure for a node. (This should really be a method of the `FSA` class, but it's easier to structure this assignment if it's an autonomous function.)

In [ ]:
def eclosure(fsa,node):
    '''computes the e-closure for a node
    args:
        fsa: an FSA
        node: a specific node
    returns:
        a set of nodes that can be reached only by
        following only e-arcs
    '''
    # YOUR CODE HERE
    closure = set()  
    stack = [node]  
    
    while stack:
        current_node = stack.pop()  
        if current_node not in closure: 
            closure.add(current_node)
            for arc in fsa.arcs:
                if arc[0] == current_node and arc[1] == '':  
                    stack.append(arc[2]) 
    return closure

In [ ]:
fsa9 = FSA(
    start = {0},
    final = {2},
    arcs = {
        (0,'',1),
        (1,'',2),
        (0,'a',1),
        (2,'b',2)
    }
)
assert eclosure(fsa9,2) == {2}

In [ ]:
assert eclosure(fsa9,0) == {0,1,2}

In [ ]:
assert eclosure(fsa9,1) == {1,2}

5. Now we'll write a function that determinizes an FSA. It should, of course, also remove any $\epsilon$-arcs. It's convenient to have a powerset function to use for this, so here that is. Note that this operates on lists and produces a list of lists. You'll ultimately need to apply this to a set and produce a set of sets.

In [ ]:
def powerset(lst):
    if len(lst) == 0:
        return [[]]
    subsets = []
    first = lst[0]
    rest = lst[1:]
    for subset in powerset(rest):
        subsets.append(subset)
        subsets.append(subset[:] + [first])
    return subsets

Now we have the `determinize()` function:

In [ ]:
def determinize(fsa):
    '''determinizes an FSA
    args:
        fsa: the FSA to determinize
    returns:
        a new determinized FSA
    '''
    # YOUR CODE HERE
    def eclosure(states):
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            for arc in fsa.arcs:
                if arc[0] == state and arc[1] == '':
                    if arc[2] not in closure:
                        closure.add(arc[2])
                        stack.append(arc[2])
        return closure

    alphabet = {arc[1] for arc in fsa.arcs if arc[1] != ''}
    
    start_closure = eclosure(fsa.start)
    
    new_start = frozenset(start_closure)
    unprocessed = [new_start]
    processed = set()
    new_arcs = set()
    new_final = set()

    state_mapping = {}
    state_counter = 0
    state_mapping[new_start] = state_counter
    state_counter += 1

    while unprocessed:
        current_state = unprocessed.pop()
        processed.add(current_state)

        if any(state in fsa.final for state in current_state):
            new_final.add(state_mapping[current_state])

        for symbol in alphabet:
            next_state = set()
            for state in current_state:
                for arc in fsa.arcs:
                    if arc[0] == state and arc[1] == symbol:
                        next_state.update(eclosure({arc[2]}))

            if next_state:
                frozen_next_state = frozenset(next_state)
                if frozen_next_state not in state_mapping:
                    state_mapping[frozen_next_state] = state_counter
                    state_counter += 1
                    unprocessed.append(frozen_next_state)
                new_arcs.add((state_mapping[current_state], symbol, state_mapping[frozen_next_state]))

    return FSA(
        start={state_mapping[new_start]},
        final=new_final,
        arcs=new_arcs
    )

In [ ]:
fsa10 = FSA(
    start={0},
    final={2},
    arcs={
        (0,'a',0),
        (0,'b',1),
        (1,'b',1),
        (1,'b',2)
    }
)
fsa11 = determinize(fsa10)
assert type(fsa11) == FSA

In [ ]:
alphabet = 'ab'
strings = [s1 + s2 + s3 + s4 \
           for s1 in alphabet \
           for s2 in alphabet \
           for s3 in alphabet \
           for s4 in alphabet
          ]
results = []
for s in strings:
    results.append(fsa10.accept(s) == fsa11.accept(s))
assert all(results)